In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Kaggle Mini GPT-2 中文预训练

**📋 目标规格**

指标	目标值

模型架构	GPT-2 (6 层, 768 hidden, 12 heads) ≈ 67.5M 参数

训练数据	中文维基百科 + 知乎 (~625M tokens)

Context 长度	1024 tokens

训练环境	Kaggle Notebook, GPU T4 x2, 12h 限时

最终 Loss	< 4.0 (理想 < 3.5)

部署平台	HuggingFace Spaces (Gradio, CPU)

**核心优化**

优化项	               收益	           代码位置

torch.compile()	   速度 +50-100%	       Cell 5 模型初始化

SDPA Attention	   速度 +10-30%	       Cell 5 config

Liger LayerNorm	  LayerNorm +30%	   Cell 5 patch

Liger FusedLinearCrossEntropy	显存 -6080%, 速度 +50100%	Cell 7 LigerTrainer

8-bit AdamW	       优化器显存 -75%	   Cell 7 训练参数

数据加载优化	       吞吐量 +10-30%	   Cell 7 训练参数

FP16 混合精度	    显存减半	           Cell 7 训练参数

**T4 硬件限制**

❌ 不支持 BF16（只能用 FP16）

❌ 不支持 Flash Attention 2.x

✅ 支持 Triton kernels (Liger Kernel)

✅ 每卡 15GB，优化后可用更多

# 🧱 第一阶段：环境配置与数据准备

**1.1 Kaggle Notebook 初始化**

在 Kaggle 新建 Notebook 时：

Accelerator: GPU T4 x2

Internet: On

Persistence: Enable Files persistence (重要！)

In [8]:
# === Cell 1: 环境诊断与库安装 ===
import os
# 启用双 GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
# 检查 GPU 可用性
!nvidia-smi
# 安装依赖（不指定版本，使用 Kaggle 预装的兼容版本）
!pip install -q \
    transformers datasets accelerate \
    huggingface_hub sentencepiece tokenizers \
    bitsandbytes \
    liger-kernel
# 验证安装
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
# 注意：Kaggle 环境不支持 DDP，Trainer 会自动使用 DataParallel

Tue Feb  3 05:22:39 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

TIP
重要：不要指定版本号！Kaggle 环境会自动选择兼容版本。

**1.2 HuggingFace 登录**

In [10]:
# === Cell 2: HuggingFace Hub 登录 ===
from huggingface_hub import notebook_login, HfFolder
# 方法 1: 交互式登录 (需要手动输入 token)
notebook_login()
# 方法 2: 非交互式 (用于 "Save & Run All" 模式)
# import os
# os.environ["HF_TOKEN"] = "hf_your_token_here"
# HfFolder.save_token(os.environ["HF_TOKEN"])
# 验证登录状态
from huggingface_hub import whoami
try:
    user_info = whoami()
    HF_USERNAME = user_info["name"]
    print(f"✅ 已登录为: {HF_USERNAME}")
except Exception as e:
    print(f"❌ 登录失败: {e}")
    HF_USERNAME = "your_username"  # 需手动填写

✅ 已登录为: Wilsonwin


**IMPORTANT**
关键提示: Token 必须具有 Write 权限才能上传模型。在 HuggingFace Settings → Access Tokens 中创建。

**1.3 数据获取与预处理(维基百科 + 知乎)**

In [11]:
# === Cell 3: 多数据集加载（推荐）===
from datasets import load_dataset, concatenate_datasets, Dataset
import os
print("📥 正在下载多个数据集...")
# === 1. 加载维基百科（完整）===
print("   [1/2] 加载维基百科...")
wiki = load_dataset(
    "pleisto/wikipedia-cn-20230720-filtered",
    split="train",  # 完整数据
)
print(f"   ✅ 维基百科: {len(wiki)} 条")
# === 2. 加载知乎高赞回答 ===
print("   [2/2] 加载知乎...")
zhihu = load_dataset(
    "wangrui6/Zhihu-KOL",
    split="train",
)
print(f"   ✅ 知乎: {len(zhihu)} 条")
# === 3. 统一字段名并合并 ===
# 维基百科字段: completion
# 知乎字段: INSTRUCTION, RESPONSE (需要合并)
def process_wiki(example):
    return {"text": example["completion"]}
def process_zhihu(example):
    # 合并问题和回答
    text = f"{example['INSTRUCTION']}\n{example['RESPONSE']}"
    return {"text": text}
print("🔄 正在处理数据格式...")
wiki_processed = wiki.map(process_wiki, remove_columns=wiki.column_names, num_proc=4)
zhihu_processed = zhihu.map(process_zhihu, remove_columns=zhihu.column_names, num_proc=4)
# 合并数据集
dataset = concatenate_datasets([wiki_processed, zhihu_processed])
dataset = dataset.shuffle(seed=42)
print(f"\n✅ 数据集合并完成!")
print(f"   维基百科: {len(wiki)} 条")
print(f"   知乎: {len(zhihu)} 条")
print(f"   合计: {len(dataset)} 条")
print(f"   示例: {dataset[0]['text'][:100]}...")
# === 4. 导出为纯文本供分词器训练（优化版）===
CORPUS_FILE = "/kaggle/working/combined_corpus.txt"
print("📝 正在导出纯文本（优化版）...")
# 使用 map + 批量写入，比 for 循环快 5-10 倍
def filter_short(example):
    return len(example["text"].strip()) > 50
# 过滤短文本
filtered = dataset.filter(filter_short, num_proc=4)
print(f"   过滤后: {len(filtered)} 条 (原 {len(dataset)} 条)")
# 批量写入文件
BATCH_SIZE = 10000
with open(CORPUS_FILE, "w", encoding="utf-8") as f:
    for i in range(0, len(filtered), BATCH_SIZE):
        batch = filtered[i:i+BATCH_SIZE]["text"]
        f.write("\n".join(text.strip() for text in batch) + "\n")
        if (i // BATCH_SIZE) % 10 == 0:
            print(f"   已导出 {min(i+BATCH_SIZE, len(filtered))}/{len(filtered)}...")
# 统计语料规模
file_size_mb = os.path.getsize(CORPUS_FILE) / (1024 * 1024)
with open(CORPUS_FILE, "r", encoding="utf-8") as f:
    line_count = sum(1 for _ in f)
print(f"✅ 语料导出完成: {file_size_mb:.1f} MB, {line_count} 行")

📥 正在下载多个数据集...
   [1/2] 加载维基百科...


README.md: 0.00B [00:00, ?B/s]

wikipedia-cn-20230720-filtered.json:   0%|          | 0.00/524M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

   ✅ 维基百科: 254547 条
   [2/2] 加载知乎...


...

Map (num_proc=4):   0%|          | 0/254547 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1006218 [00:00<?, ? examples/s]


✅ 数据集合并完成!
   维基百科: 254547 条
   知乎: 1006218 条
   合计: 1260765 条
   示例: 如何看待腾讯将 Steam 网站恶意列为危险网站？
不是这个网站对你有危险，是你上这个网站对腾讯有危险，所以它说的没错...
📝 正在导出纯文本（优化版）...


Filter (num_proc=4):   0%|          | 0/1260765 [00:00<?, ? examples/s]

   过滤后: 1137084 条 (原 1260765 条)
   已导出 10000/1137084...
   已导出 110000/1137084...
   已导出 210000/1137084...
   已导出 310000/1137084...
   已导出 410000/1137084...
   已导出 510000/1137084...
   已导出 610000/1137084...
   已导出 710000/1137084...
   已导出 810000/1137084...
   已导出 910000/1137084...
   已导出 1010000/1137084...
   已导出 1110000/1137084...
✅ 语料导出完成: 2451.3 MB, 5502872 行


**WARNING**

Kaggle 限制注意事项：

合并数据集约 2.4GB，下载需要 10-15 分钟
训练时间约 8-10 小时，接近 12 小时限制
建议启用 hub_strategy="checkpoint" 确保中途保存

# 🔤 第二阶段：SentencePiece 分词器训练

**2.1 为什么选择 SentencePiece Unigram**

特性	ByteLevelBPE	SentencePiece Unigram

中文字符处理	按 UTF-8 字节切分 (膨胀 3x)	直接处理 Unicode

词表效率	低 (需要大词表)	高

序列长度	长	短 (~30-50% 压缩)

无空格语言支持	一般	优秀 (专门设计)

**2.2 分词器训练**

In [12]:
# === Cell 4: 训练 SentencePiece 分词器（优化版）===
import sentencepiece as spm
import os
import random
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing
MODEL_PREFIX = "/kaggle/working/chinese_sp"
VOCAB_SIZE = 32000
SAMPLE_SIZE = 500000  # 50 万句训练词表
# === 多进程采样导出语料 ===
print(f"📝 采样 {SAMPLE_SIZE} 句用于分词器训练...")
print(f"   CPU 核心数: {multiprocessing.cpu_count()}")
random.seed(42)
indices = list(range(len(dataset)))
random.shuffle(indices)
sampled_indices = indices[:SAMPLE_SIZE]
# 使用 datasets 的 select + map 并行处理
sampled_dataset = dataset.select(sampled_indices)
def filter_text(example):
    text = example["text"].strip()
    if 50 < len(text) < 5000:
        return {"text": text, "valid": True}
    return {"text": "", "valid": False}
# 多进程处理
print("🔄 并行处理文本...")
processed = sampled_dataset.map(
    filter_text,
    num_proc=multiprocessing.cpu_count(),
    desc="Processing"
)
# 过滤有效文本并写入
valid_texts = [ex["text"] for ex in processed if ex["valid"]]
print(f"✅ 有效文本: {len(valid_texts)} 句")
with open(CORPUS_FILE, "w", encoding="utf-8") as f:
    f.write("\n".join(valid_texts))
corpus_size_mb = os.path.getsize(CORPUS_FILE) / 1024 / 1024
print(f"✅ 语料导出完成: {corpus_size_mb:.1f} MB")
# 训练分词器
import time
print("=" * 50)
print("🔤 开始训练分词器...")
print(f"   词表大小: {VOCAB_SIZE}")
print(f"   训练句数: {len(valid_texts)}")
print(f"   CPU 线程: {os.cpu_count() or 4}")
print("   ⏳ 预计耗时 2-5 分钟，请耐心等待...")
print("=" * 50)
start_time = time.time()
spm.SentencePieceTrainer.train(
    input=CORPUS_FILE,
    model_prefix=MODEL_PREFIX,
    vocab_size=VOCAB_SIZE,
    model_type="unigram",
    character_coverage=0.9995,
    
    # === 速度优化参数 ===
    input_sentence_size=500000,
    shuffle_input_sentence=True,
    max_sentence_length=1024,
    train_extremely_large_corpus=False,
    
    # 特殊 Token
    pad_id=0, unk_id=1, bos_id=2, eos_id=3,
    pad_piece="<pad>", unk_piece="<unk>",
    bos_piece="<s>", eos_piece="</s>",
    
    # 性能优化
    num_threads=os.cpu_count() or 4,
    normalization_rule_name="nmt_nfkc_cf",
    split_by_unicode_script=True,
    split_by_number=True,
)
elapsed = time.time() - start_time
print("=" * 50)
print(f"✅ 分词器训练完成! 耗时: {elapsed:.1f} 秒")
print(f"   模型文件: {MODEL_PREFIX}.model")
print(f"   词表文件: {MODEL_PREFIX}.vocab")
# 验证分词器
sp = spm.SentencePieceProcessor(model_file=f"{MODEL_PREFIX}.model")
test_text = "人工智能正在改变世界"
tokens = sp.encode(test_text, out_type=str)
print(f"   测试分词: {test_text} → {tokens}")

📝 采样 500000 句用于分词器训练...
   CPU 核心数: 4
🔄 并行处理文本...


Processing (num_proc=4):   0%|          | 0/500000 [00:00<?, ? examples/s]

✅ 有效文本: 442856 句
✅ 语料导出完成: 796.0 MB
🔤 开始训练分词器...
   词表大小: 32000
   训练句数: 442856
   CPU 线程: 4
   ⏳ 预计耗时 2-5 分钟，请耐心等待...
✅ 分词器训练完成! 耗时: 192.1 秒
   模型文件: /kaggle/working/chinese_sp.model
   词表文件: /kaggle/working/chinese_sp.vocab
   测试分词: 人工智能正在改变世界 → ['▁', '人工智能', '正在', '改变', '世界']


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: /kaggle/working/combined_corpus.txt
  input_format: 
  model_prefix: /kaggle/working/chinese_sp
  model_type: UNIGRAM
  vocab_size: 32000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 500000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 1024
  num_threads: 4
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 1
  bos_id: 2
  eos_id: 3
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:

WARNING

常见错误: sentencepiece_trainer.cc:177 Vocab size too small 原因: 语料太小无法支撑大词表 解决: 降低 vocab_size 到 16000 或增加语料

**2.3 转换为 HuggingFace 格式**

In [15]:
# === Cell 5: 转换分词器为 HuggingFace 格式（带缓存检测）===
import sentencepiece as spm
from transformers import PreTrainedTokenizerFast, LlamaTokenizerFast, AutoTokenizer
import json
import os
import shutil

TOKENIZER_DIR = "/kaggle/working/tokenizer"

# === 缓存检测：如果 HF 格式分词器已存在则跳过 ===
if os.path.exists(f"{TOKENIZER_DIR}/tokenizer.json"):
    print("=" * 50)
    print("✅ 检测到已转换的 HuggingFace 分词器，跳过转换")
    print(f"   目录: {TOKENIZER_DIR}")
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_DIR)
    print(f"   词表大小: {len(tokenizer)}")
    # 测试分词
    test_text = "人工智能是未来科技发展的重要方向。"
    tokens = tokenizer.tokenize(test_text)
    print(f"   测试: {test_text[:20]}... → {tokens[:5]}...")
    print("=" * 50)
else:
    # === 转换分词器 ===
    os.makedirs(TOKENIZER_DIR, exist_ok=True)
    
    # 复制模型文件
    shutil.copy(f"{MODEL_PREFIX}.model", f"{TOKENIZER_DIR}/spiece.model")
    shutil.copy(f"{MODEL_PREFIX}.vocab", f"{TOKENIZER_DIR}/spiece.vocab")
    
    # 创建 tokenizer_config.json
    tokenizer_config = {
        "bos_token": "<s>",
        "eos_token": "</s>",
        "unk_token": "<unk>",
        "pad_token": "<pad>",
        "sp_model_kwargs": {},
        "add_bos_token": False,
        "add_eos_token": False,
        "model_max_length": 512,
        "tokenizer_class": "PreTrainedTokenizerFast"
    }
    with open(f"{TOKENIZER_DIR}/tokenizer_config.json", "w") as f:
        json.dump(tokenizer_config, f, indent=2, ensure_ascii=False)
    
    # 创建 special_tokens_map.json
    special_tokens = {
        "bos_token": "<s>",
        "eos_token": "</s>",
        "unk_token": "<unk>",
        "pad_token": "<pad>"
    }
    with open(f"{TOKENIZER_DIR}/special_tokens_map.json", "w") as f:
        json.dump(special_tokens, f, indent=2, ensure_ascii=False)
    
    # 使用 LlamaTokenizerFast 加载 SP 模型
    tokenizer = LlamaTokenizerFast(
        vocab_file=f"{MODEL_PREFIX}.model",
        bos_token="<s>",
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        add_bos_token=False,
        add_eos_token=True,
    )
    
    # 保存为标准 HuggingFace 格式
    tokenizer.save_pretrained(TOKENIZER_DIR)
    print(f"✅ 分词器转换完成!")
    print(f"   目录: {TOKENIZER_DIR}")
    print(f"   词表大小: {len(tokenizer)}")
    
    # === 上传分词器到 HuggingFace Hub ===
    TOKENIZER_REPO = f"{HF_USERNAME}/chinese-sp-32k"
    try:
        tokenizer.push_to_hub(TOKENIZER_REPO)
        print(f"\n🚀 分词器已上传至: https://huggingface.co/{TOKENIZER_REPO}")
        print(f"   使用方式: AutoTokenizer.from_pretrained('{TOKENIZER_REPO}')")
    except Exception as e:
        print(f"⚠️ 上传失败: {e}")
        print("   可稍后手动上传: tokenizer.push_to_hub('你的仓库名')")
    
    # 测试分词效果
    test_text = "人工智能是未来科技发展的重要方向。"
    tokens = tokenizer.tokenize(test_text)
    ids = tokenizer.encode(test_text)
    print(f"\n测试分词:")
    print(f"   原文: {test_text}")
    print(f"   Tokens: {tokens}")
    print(f"   IDs: {ids}")
    print(f"   解码: {tokenizer.decode(ids)}")

✅ 检测到已转换的 HuggingFace 分词器，跳过转换
   目录: /kaggle/working/tokenizer
   词表大小: 32000
   测试: 人工智能是未来科技发展的重要方向。... → ['人工智能', '是', '未来', '科技', '发展']...


**CAUTION**

关键检查点: 确保 tokenizer.decode(tokenizer.encode(text)) == text 如果解码结果与原文不一致，说明分词器配置有问题。

# 🧠 第三阶段：模型初始化与数据管道

**3.1 模型架构配置**

In [16]:
# === Cell 6: GPT-2 模型配置与初始化 ===
from transformers import GPT2Config, GPT2LMHeadModel
# 重新加载分词器 (确保使用最新保存的版本)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_DIR)
# === 模型配置（优化版）===
config = GPT2Config(
    vocab_size=len(tokenizer),
    
    # === 架构（优化后）===
    n_positions=1024,                   # 增大 context（显存优化后支持）
    n_ctx=1024,
    n_embd=768,
    n_layer=6,
    n_head=12,
    n_inner=3072,
    activation_function="gelu_new",
    
    # === 正则化（预训练阶段关闭 dropout）===
    resid_pdrop=0.0,
    embd_pdrop=0.0,
    attn_pdrop=0.0,
    
    # === 特殊 Token ID ===
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)
# === 优化 0: SDPA Attention ===
try:
    config._attn_implementation = "sdpa"
    print("✅ 已设置 SDPA attention（Memory-Efficient 后端）")
except:
    pass
# 初始化模型 (随机权重)
model = GPT2LMHeadModel(config)
print(f"✅ 参数量: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")
# === 优化 1: Liger Kernel LayerNorm（+30% 速度）===
try:
    from liger_kernel.transformers import LigerLayerNorm
    import torch.nn as nn
    
    def patch_layernorm(model):
        patched_count = 0
        for name, module in list(model.named_modules()):
            if isinstance(module, nn.LayerNorm):
                parent_name = '.'.join(name.split('.')[:-1])
                child_name = name.split('.')[-1]
                parent = model.get_submodule(parent_name) if parent_name else model
                liger_ln = LigerLayerNorm(module.normalized_shape, eps=module.eps)
                liger_ln.weight = module.weight
                if module.bias is not None:
                    liger_ln.bias = module.bias
                setattr(parent, child_name, liger_ln)
                patched_count += 1
        return model, patched_count
    
    model, count = patch_layernorm(model)
    print(f"✅ 已替换 {count} 个 LayerNorm 为 Liger 版本")
except ImportError:
    print("⚠️ liger-kernel 未安装，跳过 LayerNorm 优化")
# 保存 lm_head.weight（供 FusedLinearCrossEntropy 使用）
LM_HEAD_WEIGHT = model.lm_head.weight

# === 优化 2: torch.compile ===
# 注意：torch.compile 和 DataParallel 不兼容，选择 DataParallel（使用双 GPU）
# 如果只有单 GPU，可以启用 torch.compile
USE_COMPILE = False  # DataParallel 模式下必须禁用
if USE_COMPILE and torch.cuda.device_count() == 1:
    try:
        model = torch.compile(model)
        print("✅ 已启用 torch.compile")
    except Exception as e:
        print(f"⚠️ torch.compile 不可用: {e}")

# === 优化 3: Gradient Checkpointing（在 DataParallel 之前启用）===
# 必须在包装 DataParallel 之前启用，否则 Trainer 会报错
model.gradient_checkpointing_enable()
print("✅ 已启用 Gradient Checkpointing")

# 强制使用 DataParallel（双 GPU）
if torch.cuda.device_count() > 1:
    print(f"🎮 启用 DataParallel，使用 {torch.cuda.device_count()} 个 GPU")
    model = torch.nn.DataParallel(model)

2026-02-03 05:30:28.112087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770096628.297556      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770096628.347656      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770096628.774592      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770096628.774619      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770096628.774622      55 computation_placer.cc:177] computation placer alr

✅ 已设置 SDPA attention（Memory-Efficient 后端）
✅ 参数量: 67.9M
✅ 已替换 13 个 LayerNorm 为 Liger 版本
✅ 已启用 Gradient Checkpointing
🎮 启用 DataParallel，使用 2 个 GPU


**TIP**

torch.compile 优化：

PyTorch 2.0+ 的计算图编译功能

自动融合算子，减少 kernel 启动开销

预计训练速度提升 50-100%

首次运行有 1-2 分钟编译开销

**3.2 数据 Tokenization 与 Packing**

In [17]:
# === Cell 7: 数据预处理与 Packing（带缓存检测）===
from datasets import Dataset, load_from_disk
import numpy as np
import multiprocessing
import time
from itertools import chain

NUM_PROC = multiprocessing.cpu_count()
BLOCK_SIZE = 1024
LM_DATASET_PATH = "/kaggle/working/lm_dataset"

# === 缓存检测：如果处理后的数据集已存在则跳过 ===
if os.path.exists(LM_DATASET_PATH):
    print("=" * 50)
    print("✅ 检测到已处理的数据集，从缓存加载")
    print(f"   路径: {LM_DATASET_PATH}")
    lm_dataset = load_from_disk(LM_DATASET_PATH)
    print(f"   样本数: {len(lm_dataset)} 个 {BLOCK_SIZE}-token 块")
    print(f"   总 Token: {len(lm_dataset) * BLOCK_SIZE / 1e9:.2f}B")
    # 验证数据
    sample = lm_dataset[0]
    print(f"   解码: {tokenizer.decode(sample['input_ids'][:50])}...")
    print("=" * 50)
else:
    print(f"🖥️ 使用 {NUM_PROC} 个 CPU 核心并行处理")
    
    # === 步骤 1: Tokenize 所有文本 ===
    print("=" * 50)
    print("🔄 正在 Tokenize...")
    tokenize_start = time.time()
    
    text_column = "text" if "text" in dataset.column_names else "completion"
    print(f"   字段: {text_column}")
    print(f"   样本数: {len(dataset)}")
    
    def tokenize_function(examples):
        return tokenizer(
            examples[text_column],
            add_special_tokens=True,
            truncation=False,
            return_attention_mask=False,
        )
    
    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        batch_size=5000,
        remove_columns=dataset.column_names,
        num_proc=NUM_PROC,
        desc="Tokenizing",
    )
    tokenize_time = time.time() - tokenize_start
    print(f"✅ Tokenization 完成: {len(tokenized_dataset)} 样本, 耗时 {tokenize_time:.1f}s")
    
    # === 步骤 2: Packing ===
    print("=" * 50)
    print("📦 正在 Packing...")
    pack_start = time.time()
    
    def group_texts(examples):
        concatenated = {k: list(chain.from_iterable(examples[k])) for k in examples.keys()}
        total_length = len(concatenated["input_ids"])
        
        if total_length >= BLOCK_SIZE:
            total_length = (total_length // BLOCK_SIZE) * BLOCK_SIZE
        
        result = {
            k: [t[i : i + BLOCK_SIZE] for i in range(0, total_length, BLOCK_SIZE)]
            for k, t in concatenated.items()
        }
        result["labels"] = result["input_ids"].copy()
        return result
    
    lm_dataset = tokenized_dataset.map(
        group_texts,
        batched=True,
        batch_size=5000,
        num_proc=NUM_PROC,
        desc="Packing",
    )
    pack_time = time.time() - pack_start
    
    # 保存到磁盘作为缓存
    lm_dataset.save_to_disk(LM_DATASET_PATH)
    
    total_time = tokenize_time + pack_time
    print("=" * 50)
    print(f"✅ 数据处理完成!")
    print(f"   Tokenize: {tokenize_time:.1f}s")
    print(f"   Packing: {pack_time:.1f}s")
    print(f"   总耗时: {total_time:.1f}s")
    print(f"   样本数: {len(lm_dataset)} 个 {BLOCK_SIZE}-token 块")
    print(f"   总 Token: {len(lm_dataset) * BLOCK_SIZE / 1e9:.2f}B")
    print(f"   已缓存至: {LM_DATASET_PATH}")
    
    # 验证数据
    sample = lm_dataset[0]
    print(f"\n📊 数据样本:")
    print(f"   解码: {tokenizer.decode(sample['input_ids'][:50])}...")

🖥️ 使用 4 个 CPU 核心并行处理
🔄 正在 Tokenize...
   字段: text
   样本数: 1260765


Tokenizing (num_proc=4):   0%|          | 0/1260765 [00:00<?, ? examples/s]

✅ Tokenization 完成: 1260765 样本, 耗时 430.9s
📦 正在 Packing...


Packing (num_proc=4):   0%|          | 0/1260765 [00:00<?, ? examples/s]

Saving the dataset (0/15 shards):   0%|          | 0/579166 [00:00<?, ? examples/s]

✅ 数据处理完成!
   Tokenize: 430.9s
   Packing: 209.4s
   总耗时: 640.3s
   样本数: 579166 个 1024-token 块
   总 Token: 0.59B
   已缓存至: /kaggle/working/lm_dataset

📊 数据样本:
   解码: 如何看待腾讯将 <pad>team 网站恶意列为危险网站<pad>不是这个网站对你有危险<pad>是你上这个网站对腾讯有危险<pad>所以它说的没错</s>你认为肖斯塔科维奇最伟大的作品是什么<pad>第十四交响曲是肖斯塔维奇晚年...


**WARNING**

Packing 注意事项:

Packing 会导致不同文章的 token 混在同一个序列中
模型会学到跨文章的错误关联
缓解方法: 在每篇文章末尾添加 <eos> token (上面已配置 add_eos_token=True)

# 🚂 第四阶段：分布式训练配置

**4.1 训练参数优化**

In [18]:
# === Cell 8: 训练参数配置 ===
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
# === 模型命名 ===
MODEL_NAME = "gpt2-chinese-mini"
REPO_ID = f"{HF_USERNAME}/{MODEL_NAME}"
# === 训练参数（显存优化版）===
NUM_EPOCHS = 2
BATCH_SIZE_PER_GPU = 16              # DataParallel 模式下降低 batch size
GRADIENT_ACCUMULATION = 2
NUM_GPUS = 2
BLOCK_SIZE = 1024
# 有效 batch size = 24 * 2 * 2 = 96 samples
# 每个 sample 1024 tokens = ~98K tokens per update
effective_batch_size = BATCH_SIZE_PER_GPU * GRADIENT_ACCUMULATION * NUM_GPUS
tokens_per_update = effective_batch_size * BLOCK_SIZE
steps_per_epoch = len(lm_dataset) // effective_batch_size
total_steps = steps_per_epoch * NUM_EPOCHS
print(f"📊 训练规模:")
print(f"   每 GPU batch: {BATCH_SIZE_PER_GPU}")
print(f"   梯度累积: {GRADIENT_ACCUMULATION}")
print(f"   有效 batch: {effective_batch_size}")
print(f"   每步 tokens: {tokens_per_update / 1e3:.1f}K")
print(f"   总步数: {total_steps}")
# === 训练参数 ===
training_args = TrainingArguments(
    output_dir=f"/kaggle/working/{MODEL_NAME}",
    overwrite_output_dir=True,
    
    # === 训练规模 ===
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE_PER_GPU,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    
    # === 学习率调度（针对大 batch 优化）===
    learning_rate=3e-4,                  # 大 batch 时降低 LR
    warmup_steps=2000,                   # 约 5% 总步数
    lr_scheduler_type="cosine",
    
    # === 优化器（8-bit 优化，显存减少 75%）===
    optim="adamw_bnb_8bit",
    weight_decay=0.1,
    adam_beta1=0.9,
    adam_beta2=0.95,
    max_grad_norm=1.0,
    
    # === 精度 ===
    fp16=True,
    bf16=False,                          # T4 不支持
    gradient_checkpointing=False,        # 已在 Cell 6 手动启用（DataParallel 前）
    
    # === 数据加载优化 ===
    dataloader_num_workers=4,
    dataloader_pin_memory=True,
    dataloader_prefetch_factor=4,
    dataloader_persistent_workers=True,
    
    # === Checkpoint ===
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,
    load_best_model_at_end=False,
    prediction_loss_only=True,    # 强制计算 eval loss
    
    # === 日志与 Hub ===
    logging_steps=1,               # 前几步每步打印，确认无误后改为 100
    logging_first_step=True,
    report_to="none",
    push_to_hub=True,
    hub_model_id=REPO_ID,
    hub_strategy="checkpoint",
    
    # === 分布式 ===
    ddp_find_unused_parameters=False,
    remove_unused_columns=False,  # torch.compile 兼容
    seed=42,
)
print(f"\n✅ 训练配置完成")
print(f"   Warmup 步数: {training_args.warmup_steps}")
print(f"   LR 调度: {training_args.lr_scheduler_type}")
print(f"   优化器: {training_args.optim}")
print(f"   保存间隔: {training_args.save_steps} 步") 

📊 训练规模:
   每 GPU batch: 16
   梯度累积: 2
   有效 batch: 64
   每步 tokens: 65.5K
   总步数: 18098

✅ 训练配置完成
   Warmup 步数: 2000
   LR 调度: SchedulerType.COSINE
   优化器: OptimizerNames.ADAMW_BNB
   保存间隔: 500 步


**TIP**
优化总结：

显存优化后 batch_size 16→24，context 512→1024

大 batch 需降低 learning_rate 6e-4→3e-4

adamw_bnb_8bit + Liger Kernel 共节省 80%+ 显存

**IMPORTANT**
Chinchilla 分析：

当前：625M tokens / 67.5M params ≈ 9 tokens/param

理想：20-25 tokens/param（需 1.3-1.6B tokens）

结论：数据略少，2 epochs 充分训练

**4.2 Data Collator**

In [19]:
# === Cell 9: Data Collator 配置 ===
# 语言模型专用 Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # 因果语言模型 (非 Masked LM)
)
# 测试 collator
test_batch = [lm_dataset[i] for i in range(4)]
collated = data_collator(test_batch)
print("📦 Data Collator 测试:")
print(f"   input_ids shape: {collated['input_ids'].shape}")
print(f"   labels shape: {collated['labels'].shape}") 

📦 Data Collator 测试:
   input_ids shape: torch.Size([4, 1024])
   labels shape: torch.Size([4, 1024])


**4.3 创建 Trainer**

In [21]:
# === Cell 10: 自定义 Trainer + 生成回调 ===
from transformers import Trainer, TrainerCallback
# 确保 LM_HEAD_WEIGHT 存在（如果之前的 Cell 没有定义）
try:
    LM_HEAD_WEIGHT
except NameError:
    # 处理 DataParallel 包装
    base_model = model.module if hasattr(model, 'module') else model
    # 处理 torch.compile 包装
    base_model = base_model._orig_mod if hasattr(base_model, '_orig_mod') else base_model
    LM_HEAD_WEIGHT = base_model.lm_head.weight
    print("✅ 已获取 LM_HEAD_WEIGHT")
# === 生成回调：每次评估时测试 prompt ===
class GenerationCallback(TrainerCallback):
    def __init__(self, tokenizer, prompts=None):
        self.tokenizer = tokenizer
        self.prompts = prompts or ["中国的历史", "人工智能是", "知乎上有人问"]
    
    def on_evaluate(self, args, state, control, model, **kwargs):
        print("\n" + "=" * 50)
        print(f"📝 Step {state.global_step} - 生成样本:")
        print("=" * 50)
        
        # 处理 DataParallel 和 torch.compile 包装
        eval_model = model
        if hasattr(model, 'module'):  # DataParallel
            eval_model = model.module
        if hasattr(eval_model, '_orig_mod'):  # torch.compile
            eval_model = eval_model._orig_mod
        
        eval_model.eval()
        device = next(eval_model.parameters()).device
        for prompt in self.prompts:
            try:
                inputs = self.tokenizer(prompt, return_tensors="pt").to(device)
                with torch.no_grad():
                    outputs = eval_model.generate(
                        **inputs, max_new_tokens=50,
                        do_sample=True, temperature=0.8, top_k=50,
                        pad_token_id=self.tokenizer.pad_token_id,
                    )
                generated = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                print(f"  [{prompt}] → {generated}")
            except Exception as e:
                print(f"  [{prompt}] → 生成失败: {e}")
        print("=" * 50 + "\n")
        eval_model.train()
# === 日志回调：前 10 步每步打印，之后每 100 步 ===
class LoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            loss = logs.get("loss", logs.get("eval_loss", None))
            if loss is not None:
                step = state.global_step
                # 前 10 步每步打印，之后每 100 步打印
                if step <= 10 or step % 100 == 0:
                    lr = logs.get("learning_rate", 0)
                    print(f"📊 Step {step}: loss={loss:.4f}, lr={lr:.2e}")
# === LigerTrainer: 使用 FusedLinearCrossEntropy ===
USE_FUSED_CE = False  # 设置为 False 可禁用
try:
    from liger_kernel.transformers import LigerFusedLinearCrossEntropyLoss
    
    class LigerTrainer(Trainer):
        """使用 FusedLinearCrossEntropy，显存 -60~80%"""
        def __init__(self, *args, lm_head_weight=None, **kwargs):
            super().__init__(*args, **kwargs)
            self.fused_loss_fn = LigerFusedLinearCrossEntropyLoss()
            self.lm_head_weight = lm_head_weight
            print("✅ LigerTrainer: 已启用 FusedLinearCrossEntropy")
        
        def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
            labels = inputs.pop("labels")
            base_model = getattr(model, '_orig_mod', model)
            transformer = getattr(base_model, 'transformer', None)
            outputs = transformer(
                input_ids=inputs["input_ids"],
                attention_mask=inputs.get("attention_mask"),
            )
            hidden_states = outputs.last_hidden_state
            shift_hidden = hidden_states[..., :-1, :].contiguous().view(-1, hidden_states.size(-1))
            shift_labels = labels[..., 1:].contiguous().view(-1)
            loss = self.fused_loss_fn(self.lm_head_weight, shift_hidden, shift_labels)
            if return_outputs:
                from transformers.modeling_outputs import CausalLMOutputWithPast
                return loss, CausalLMOutputWithPast(loss=loss)
            return loss
    
    if USE_FUSED_CE:
        TrainerClass = LigerTrainer
        trainer_kwargs = {"lm_head_weight": LM_HEAD_WEIGHT}
        print("✅ 将使用 LigerTrainer + FusedLinearCrossEntropy")
    else:
        TrainerClass = Trainer
        trainer_kwargs = {}
except ImportError:
    print("⚠️ liger-kernel 未安装，使用标准 Trainer")
    TrainerClass = Trainer
    trainer_kwargs = {}
# === 创建 Trainer ===
generation_callback = GenerationCallback(
    tokenizer=tokenizer,
    prompts=["中国的历史", "人工智能是", "知乎上有人问", "科学技术的发展"]
)
logging_callback = LoggingCallback()
trainer = TrainerClass(
    model=model,
    args=training_args,
    train_dataset=lm_dataset,
    eval_dataset=lm_dataset.select(range(min(1000, len(lm_dataset)))),
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[generation_callback, logging_callback],  # 两个回调
    **trainer_kwargs,
)
print("🎯 Trainer 创建完成!")

🎯 Trainer 创建完成!


/tmp/ipykernel_55/267342749.py:106: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = TrainerClass(


# 🚀 第五阶段：训练执行

**5.1 启动训练**

In [ ]:
# === Cell 11: 多 GPU 训练（DataParallel 模式）===
# 注意：Kaggle 环境不支持 DDP，Trainer 会自动使用 DataParallel
import time
import gc
import torch
# 清理 GPU 缓存
gc.collect()
torch.cuda.empty_cache()
print("=" * 60)
print("🚀 开始多 GPU 预训练")
print("=" * 60)
print(f"⏰ 开始时间: {time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🖥️ GPU 显存: {torch.cuda.memory_allocated() / 1024**3:.2f} GB 已用")
print(f"🎮 GPU 数量: {torch.cuda.device_count()}")
print(f"📍 模型将上传至: https://huggingface.co/{REPO_ID}")
print()
print("📈 Loss 参考:")
print("   初始 (随机权重): ~10.0 - 11.0")
print("   500 步后: ~6.0 - 7.0")
print("   收敛目标: < 4.0")
print("=" * 60)
# === 关键检查：vocab_size 一致性 ===
# 处理 DataParallel 包装
base_model = model.module if hasattr(model, 'module') else model
model_vocab = base_model.config.vocab_size
tokenizer_vocab = len(tokenizer)
print(f"\n🔍 vocab_size 检查:")
print(f"   模型: {model_vocab}")
print(f"   分词器: {tokenizer_vocab}")
if model_vocab != tokenizer_vocab:
    print(f"❌ 错误: vocab_size 不匹配! 模型={model_vocab}, 分词器={tokenizer_vocab}")
    print("   请重新运行 Cell 6 创建模型，确保使用 vocab_size=len(tokenizer)")
    raise ValueError(f"vocab_size mismatch: model={model_vocab}, tokenizer={tokenizer_vocab}")
# 检查整个数据集的 token ID 范围（抽样检查）
print("   检查数据集 token ID 范围...")
max_token_id = 0
problem_samples = []
for i in range(0, len(lm_dataset), max(1, len(lm_dataset) // 100)):  # 抽样 100 个
    sample_ids = lm_dataset[i]["input_ids"]
    sample_max = max(sample_ids)
    if sample_max > max_token_id:
        max_token_id = sample_max
    if sample_max >= model_vocab:
        problem_samples.append((i, sample_max))
print(f"   数据集最大 token ID: {max_token_id}")
print(f"   模型 vocab_size: {model_vocab}")
if problem_samples:
    print(f"❌ 发现 {len(problem_samples)} 个样本的 token ID 越界!")
    for idx, max_id in problem_samples[:5]:
        print(f"   样本 {idx}: max_token_id = {max_id} >= {model_vocab}")
    print("\n💡 解决方案:")
    print("   1. 删除 /kaggle/working/tokenizer 目录")
    print("   2. 重新运行 Cell 4 (训练分词器)")
    print("   3. 重新运行 Cell 5 (保存分词器)")
    print("   4. 重新运行 Cell 6-10")
    raise ValueError(f"Token IDs out of range!")
if max_token_id >= model_vocab:
    print(f"❌ 错误: token ID {max_token_id} >= vocab_size {model_vocab}")
    raise ValueError(f"Token ID out of range: {max_token_id} >= {model_vocab}")
print("✅ vocab_size 检查通过!\n")
# === 开始训练 ===
try:
    trainer.train()
    print("\n✅ 训练完成!")
    
    # 保存模型
    trainer.save_model()
    tokenizer.save_pretrained(f"/kaggle/working/{MODEL_NAME}")
    
    # 上传到 Hub
    print("\n📤 正在上传最终模型到 HuggingFace Hub...")
    from huggingface_hub import HfApi
    api = HfApi()
    api.upload_folder(
        folder_path=f"/kaggle/working/{MODEL_NAME}",
        repo_id=REPO_ID,
        commit_message="Training complete"
    )
    print(f"\n🎉 模型已发布至: https://huggingface.co/{REPO_ID}")
    
except KeyboardInterrupt:
    print("\n⚠️ 训练被中断，正在保存当前状态...")
    trainer.save_model()
    print("💾 模型已保存，下次可用 resume_from_checkpoint=True 继续")
    
except Exception as e:
    print(f"\n❌ 训练出错: {e}")
    print("\n💡 排错指南:")
    print("   - OOM: 减少 BATCH_SIZE_PER_GPU")
    print("   - 速度慢: 检查 gradient_checkpointing")
    print("   - Loss 不降: 检查数据和 learning_rate")
    raise

🚀 开始多 GPU 预训练
⏰ 开始时间: 2026-02-02 19:15:12
🖥️ GPU 显存: 0.28 GB 已用
🎮 GPU 数量: 2
📍 模型将上传至: https://huggingface.co/Wilsonwin/gpt2-chinese-mini

📈 Loss 参考:
   初始 (随机权重): ~10.0 - 11.0
   500 步后: ~6.0 - 7.0
   收敛目标: < 4.0

🔍 vocab_size 检查:
   模型: 32000
   分词器: 32000
   检查数据集 token ID 范围...
   数据集最大 token ID: 31991
   模型 vocab_size: 32000
✅ vocab_size 检查通过!



Step,Training Loss,Validation Loss
500,7.590900,No log
1000,6.907700,No log
1500,6.184900,No log
2000,5.664300,No log
2500,5.490800,No log
3000,5.343200,No log
3500,5.178100,No log
4000,5.098000,No log
4500,4.987400,No log
5000,4.973800,No log


📊 Step 1: loss=10.4946, lr=0.00e+00
📊 Step 2: loss=10.5037, lr=1.50e-07
📊 Step 3: loss=10.5025, lr=3.00e-07
📊 Step 4: loss=10.5013, lr=4.50e-07
📊 Step 5: loss=10.4833, lr=6.00e-07
📊 Step 6: loss=10.4815, lr=7.50e-07
📊 Step 7: loss=10.4621, lr=9.00e-07
📊 Step 8: loss=10.4456, lr=1.05e-06
📊 Step 9: loss=10.4325, lr=1.20e-06
📊 Step 10: loss=10.4055, lr=1.35e-06
📊 Step 100: loss=9.2019, lr=1.49e-05
📊 Step 200: loss=8.4251, lr=2.99e-05
📊 Step 300: loss=8.1103, lr=4.48e-05
📊 Step 400: loss=7.8093, lr=5.98e-05
📊 Step 500: loss=7.5909, lr=7.48e-05


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



📝 Step 500 - 生成样本:
  [中国的历史] → 中国的历史《》》》《》 《》《《》》》《 《《《《《《史 》》》》》》》 《《《《》》《。《《》》》》《。《《
  [人工智能是] → 人工智能是在12年的。 119的                                        
  [知乎上有人问] → 知乎上有人问中国在一张天。但是他她他在他从上他的《白》的。他他又。他他他一说。”他他”他他他我他他他他来他他他他他他他
  [科学技术的发展] → 科学技术的发展在                                                 

📊 Step 600: loss=7.5024, lr=8.98e-05
📊 Step 700: loss=7.3040, lr=1.05e-04
📊 Step 800: loss=7.1559, lr=1.20e-04
📊 Step 900: loss=6.9714, lr=1.35e-04
📊 Step 1000: loss=6.9077, lr=1.50e-04


...


📝 Step 9000 - 生成样本:
  [中国的历史] → 中国的历史为什么在 15 年没有喝完啤酒后才开始去喝中国啤酒 ?  
  [人工智能是] → 人工智能是有哪些经典的电影台词让你印象深刻?为什么是《小时代》里导演的一句台词或者台词让你印象深刻。 一、《小时代》里最经典的台词 有一部《小时代》里最经典的台词就是《大时代》
  [知乎上有人问] → 知乎上有人问《龙应台》与《龙应台》相比有何异同点?如何评价《龙应台》第二季第一集? 龙应台的造型与《龙应台》的造型相比有何异同点? 龙应台
  [科学技术的发展] → 科学技术的发展《流浪地球》中人类为什么要选择救活人为救生者或有计划的救生者去救生者。                        

📊 Step 9100: loss=4.4941, lr=1.78e-04
📊 Step 9200: loss=4.6641, lr=1.75e-04
📊 Step 9300: loss=4.5998, lr=1.72e-04
📊 Step 9400: loss=4.6729, lr=1.69e-04
📊 Step 9500: loss=4.5515, lr=1.66e-04


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



📝 Step 9500 - 生成样本:
  [中国的历史] → 中国的历史《魔戒三部曲》中甘道夫对索伦有何重大的贡献? 甘道夫是大法师出身 他是一个历史观人 他是一个大法师 魔戒三部曲是大法师 他是一个故事的开端 他是一个历史观人 
  [人工智能是] → 人工智能是美国有哪些优秀的 8 位诺贝尔奖获得者和获奖人都有过哪些优秀的背景、获得成就、获奖经历、获奖经历、获奖经历、获奖经历、获奖经历等等。             
  [知乎上有人问] → 知乎上有人问如何评价《极限挑战》第二季《极限挑战》中刘涛的台词「我宁愿吃麦当劳。」——「吃麦当劳也不愿吃麦当劳。」——「我宁愿吃麦当劳。」——「我宁愿吃麦当劳。」——「
  [科学技术的发展] → 科学技术的发展如何评价电影《三体》的原著作者刘慈欣    

📊 Step 9600: loss=4.7119, lr=1.63e-04
📊 Step 9700: loss=4.6573, lr=1.60e-04
📊 Step 9800: loss=4.6211, lr=1.57e-04
📊 Step 9900: loss=4.5487, lr=1.54e-04
📊 Step 10000: loss=4.6447, lr=1.51e-04


/usr/local/lib/python3.12/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



📝 Step 10000 - 生成样本:
  [中国的历史] → 中国的历史有哪些特别有意思的冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识冷知识
  [人工智能是] → 人工智能是你听过最难听的一首诗是什么? 「小人」
  [知乎上有人问] → 知乎上有人问中国银行间债券市场如何规范债券投资者的市场定价?市场定价?
  [科学技术的发展] → 科学技术的发展怎样评价电影《罗生门》的真假? 看原著《罗生门》的真假 看原著《罗生门》的真假 看原著原著《罗生门》的真假 看原著原著《罗生门》真假 看原著《罗生门》真假



In [3]:
from huggingface_hub import snapshot_download

# 1. 设置你的仓库ID
REPO_ID = "Wilsonwin/gpt2-chinese-mini" 

# 2. 如果你的文件像截图中那样，是在 'last-checkpoint' 这个子文件夹里：
#    我们需要把这个文件夹的内容下载下来
local_checkpoint_path = "/kaggle/working/downloaded_checkpoint"

print("⏳ 正在从 Hugging Face 下载 Checkpoint...")

# 下载整个仓库（或者指定文件夹）
snapshot_download(
    repo_id=REPO_ID,
    local_dir=local_checkpoint_path,
    # 如果截图里的文件确实是在仓库根目录下的 last-checkpoint 文件夹里，加上下面这行：
    # allow_patterns=["last-checkpoint/*"], 
    # 但通常如果是 trainer push 的，直接下载整个 repo 根目录即可，不需要 allow_patterns
)

print(f"✅ 下载完成！路径位于: {local_checkpoint_path}")

⏳ 正在从 Hugging Face 下载 Checkpoint...


Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

last-checkpoint/model.safetensors:   0%|          | 0.00/272M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

last-checkpoint/optimizer.pt:   0%|          | 0.00/290M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

training_args.bin:   0%|          | 0.00/5.84k [00:00<?, ?B/s]

✅ 下载完成！路径位于: /kaggle/working/downloaded_checkpoint


**CAUTION**

断点续训: 如果训练中断，下次运行时使用

In [22]:
# 1. 确认路径：你需要指向包含 config.json 和 optimizer.pt 的那一层文件夹
# 如果你之前是用 snapshot_download 下载的，可能需要检查一下是否多了一层文件夹
import os

# 假设这是你刚才下载的目标目录
chk_path = "/kaggle/working/downloaded_checkpoint"

# 🛑 检查点：防止文件夹套娃
# 有时候下载下来是 /kaggle/working/downloaded_checkpoint/last-checkpoint/
# 或者是 /kaggle/working/downloaded_checkpoint/checkpoint-10000/
# 下面这段代码帮你自动找到包含 config.json 的那一层
if "config.json" not in os.listdir(chk_path):
    for root, dirs, files in os.walk(chk_path):
        if "config.json" in files:
            chk_path = root
            break

print(f"🎯 最终锁定的 Checkpoint 路径: {chk_path}")

# 2. 开始续训
# 注意：这里不用 True，而是直接用路径字符串
trainer.train(resume_from_checkpoint=chk_path)

🎯 最终锁定的 Checkpoint 路径: /kaggle/working/downloaded_checkpoint


AttributeError: 'DataParallel' object has no attribute '_keys_to_ignore_on_save'

**5.2 训练监控与诊断**

In [ ]:
# === Cell 12: 训练诊断 (可选，在训练过程中运行) ===
# 查看训练日志
import json
log_history = trainer.state.log_history
if log_history:
    print("📊 训练历史:")
    for log in log_history[-5:]:  # 最近 5 条
        if "loss" in log:
            print(f"   Step {log.get('step', 'N/A')}: Loss = {log['loss']:.4f}")
# 绘制 Loss 曲线 (需要 matplotlib)
try:
    import matplotlib.pyplot as plt
    
    losses = [log["loss"] for log in log_history if "loss" in log]
    steps = [log["step"] for log in log_history if "loss" in log]
    
    plt.figure(figsize=(10, 4))
    plt.plot(steps, losses)
    plt.xlabel("Steps")
    plt.ylabel("Loss")
    plt.title("Training Loss Curve")
    plt.grid(True)
    plt.savefig("/kaggle/working/loss_curve.png", dpi=100)
    plt.show()
except ImportError:
    print("matplotlib 未安装，跳过绘图")

# 🌐 第六阶段：HuggingFace Spaces 部署

**6.1 创建 Space**

访问 huggingface.co/spaces

点击 Create new Space

配置:

Space name: gpt2-chinese-demo

SDK: Gradio

Hardware: CPU basic (免费)

创建后进入 Files tab

**6.2 app.py**

In [ ]:
# === HuggingFace Space: app.py ===
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# 🔴 修改为您的模型 ID
MODEL_ID = "YOUR_USERNAME/gpt2-chinese-mini"
print(f"正在加载模型: {MODEL_ID}")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    model = AutoModelForCausalLM.from_pretrained(MODEL_ID)
    model.eval()
    print("✅ 模型加载成功!")
except Exception as e:
    print(f"❌ 模型加载失败: {e}")
    tokenizer = None
    model = None
def generate_text(
    prompt: str,
    max_length: int = 150,
    temperature: float = 0.8,
    top_k: int = 50,
    top_p: float = 0.95,
    num_return_sequences: int = 1,
) -> str:
    """生成文本"""
    if model is None or tokenizer is None:
        return "❌ 模型未加载成功，请检查模型 ID"
    
    if not prompt.strip():
        return "请输入提示文本"
    
    try:
        inputs = tokenizer(prompt, return_tensors="pt")
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=max_length,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                num_return_sequences=num_return_sequences,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                no_repeat_ngram_size=2,  # 减少重复
            )
        
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return generated_text
    
    except Exception as e:
        return f"生成出错: {e}"
# === Gradio 界面 ===
with gr.Blocks(title="中文 GPT-2 Mini", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🇨🇳 中文 GPT-2 Mini
    
    这是一个在 Kaggle 上从零预训练的中文语言模型。
    
    > ⚠️ **注意**: 这是一个小型演示模型，生成质量有限，可能产生不准确或无意义的内容。
    """)
    
    with gr.Row():
        with gr.Column(scale=2):
            prompt_input = gr.Textbox(
                label="输入提示",
                placeholder="例如：人工智能的未来发展",
                lines=3,
            )
            
            with gr.Row():
                max_length = gr.Slider(50, 300, value=150, step=10, label="最大长度")
                temperature = gr.Slider(0.1, 1.5, value=0.8, step=0.1, label="温度")
            
            with gr.Row():
                top_k = gr.Slider(10, 100, value=50, step=5, label="Top-K")
                top_p = gr.Slider(0.5, 1.0, value=0.95, step=0.05, label="Top-P")
            
            generate_btn = gr.Button("🚀 生成", variant="primary")
        
        with gr.Column(scale=3):
            output = gr.Textbox(label="生成结果", lines=10)
    
    generate_btn.click(
        fn=generate_text,
        inputs=[prompt_input, max_length, temperature, top_k, top_p],
        outputs=output,
    )
    
    gr.Examples(
        examples=[
            ["中国的历史"],
            ["人工智能是"],
            ["今天天气"],
            ["科学技术的发展"],
        ],
        inputs=prompt_input,
    )
demo.launch()

**6.3 requirements.txt**

In [ ]:
transformers>=4.37.0
torch>=2.0.0
gradio>=4.0.0
sentencepiece